In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
pd.set_option('display.max_columns', None)
import os
import tempfile
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Model and performance evaluation
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
lbl = preprocessing.LabelEncoder()
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import precision_recall_fscore_support as score
# Hyperparameter tuning
from sklearn.model_selection import StratifiedKFold, cross_val_score, GridSearchCV, RandomizedSearchCV
from hyperopt import tpe, STATUS_OK, Trials, hp, fmin, STATUS_OK, space_eval

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Crossroads/engg_data/crossroads_full_feature_set.csv')

In [ ]:
ensemble_validation=pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/Ensemble_Validation_DataSet_v1.csv')

In [ ]:
ensemble_validation = ensemble_validation.drop(["Unnamed: 0"], 1)

In [ ]:
ensemble_validation.head()

,UniqueID,isAttended_Yes,class_name,count,%Attended,event_name_count
0,CLT21LAR-417-6-20,1,OPEN,8.0,100.000000,2.0
1,CLT21PRE-644-19-16,0,OPEN,1216.0,87.171053,18.0
2,CLT21LAR-411-9-1,0,OPEN,4.0,50.000000,2.0
3,CLT22JAX-417-1-9,1,OPEN,32.0,96.875000,16.0
4,CLT21SEA-423-7-6,1,OPEN,486.0,75.720165,9.0


In [ ]:
Ayan_CB_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_CatBoost_v1.2_val_preds.csv')
Ayan_NN_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_NN_v1.0_TE_val_preds.csv')
Ayan_XGB_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_XGB_v1.3_TE_val_preds.csv')
Ayan_LGB_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.1_TE_val.csv')
Ayan_LGB_val_AccID = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.2_TE_Acc_ID_val.csv')
# Shikhar_XGB_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Shikhar_XGB_w_dummy.csv')
Yamini_smote_val = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Yamini_XGB_Smote_val.csv')


In [ ]:
set(ensemble_validation.UniqueID.tolist()) - set(Yamini_smote_val.UniqueID.tolist()) 

set()

In [ ]:
Ayan_LGB_val_AccID.isnull().sum()

UniqueID           0
XGB_model_preds    0
dtype: int64

In [ ]:
Ayan_LGB_val.rename(columns={'XGB_model_preds':'LGBM'},inplace=True)
Ayan_LGB_val_AccID.rename(columns={'XGB_model_preds':'LGBM_AccID'},inplace=True)

In [ ]:
ensemble_validation=ensemble_validation.merge(
    Ayan_CB_val[['UniqueID','CatBoost_model_preds']], how='left', on='UniqueID').merge(
        Ayan_NN_val[['UniqueID','model_preds']], how='left', on='UniqueID').merge(
            Ayan_XGB_val[['UniqueID','XGB_model_preds']], how='left', on='UniqueID').merge(
                Ayan_LGB_val[['UniqueID','LGBM']], how='left', on='UniqueID').merge(
                    Ayan_LGB_val_AccID[['UniqueID','LGBM_AccID']], how='left' , on='UniqueID').merge(
                            Yamini_smote_val[['UniqueID','Yamini_Smote_proba']], how='left', on='UniqueID')
                      

In [ ]:
ensemble_validation.head(3)

,UniqueID,isAttended_Yes,class_name,count,%Attended,event_name_count,CatBoost_model_preds,model_preds,XGB_model_preds,LGBM,LGBM_AccID,Yamini_Smote_proba
0,CLT21LAR-417-6-20,1,OPEN,8.0,100.000000,2.0,0.877009,0.865320,0.999988,0.873752,0.999957,0.865867
1,CLT21PRE-644-19-16,0,OPEN,1216.0,87.171053,18.0,0.234931,0.242824,0.484125,0.277639,0.431342,0.208784
2,CLT21LAR-411-9-1,0,OPEN,4.0,50.000000,2.0,0.862773,0.846407,0.504230,0.879845,0.498084,0.834916


In [ ]:
ensemble_validation.rename(columns = {'CatBoost_model_preds':'Catboost',
                                      'model_preds':'NN',
                                      'XGB_model_preds':'XGB',
                                      'y_proba':'XGB_dummy',
                                      'Yamini_Smote_proba':'Smote'}, inplace = True)

In [ ]:
ensemble_validation.head(1)

,UniqueID,isAttended_Yes,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LAR-417-6-20,1,OPEN,8.0,100.0,2.0,0.877009,0.86532,0.999988,0.873752,0.999957,0.865867


In [ ]:
Ayan_CB_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_CatBoost_v1.2_test_preds.csv')
Ayan_NN_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_NN_v1.0_TE_test_preds.csv')
Ayan_XGB_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Ayan_XGB_v1.3_TE_test_preds.csv')
Ayan_LGB_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.1_test.csv')
Ayan_LGB_test_AccID = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/LGB_v1.2_TE_Acc_ID_test.csv')
# Shikhar_XGB_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Shikhar_XGB_w_dummy.csv')
Yamini_smote_test = pd.read_csv('/content/drive/MyDrive/Crossroads/Ensemble_Model/saved_preds/Yamini_XGB_Smote_test.csv')


In [ ]:
Ayan_CB_test.shape

(128688, 2)

In [ ]:
Ayan_CB_val.head()

,UniqueID,CatBoost_model_preds
0,CLT21HOU-101-10-13,0.865624
1,CLT21HOU-101-10-14,0.865624
2,CLT21HOU-101-10-17,0.760997
3,CLT21HOU-101-10-5,0.927807
4,CLT21HOU-101-10-6,0.867801


In [ ]:
Ayan_LGB_test.rename(columns={'XGB_model_preds':'LGBM'},inplace=True)
Ayan_LGB_test_AccID.rename(columns={'XGB_model_preds':'LGBM_AccID'},inplace=True)

In [ ]:
ensemble_test=Ayan_NN_test.merge(
        Ayan_CB_test[['UniqueID','CatBoost_model_preds']], how='inner', on='UniqueID').merge(
            Ayan_XGB_test[['UniqueID','XGB_model_preds']], how='inner', on='UniqueID').merge(
                Ayan_LGB_test[['UniqueID','LGBM']], how='inner', on='UniqueID').merge(
                    Ayan_LGB_test_AccID[['UniqueID','LGBM_AccID']], how='inner' , on='UniqueID').merge(
                            Yamini_smote_test[['UniqueID','Yamini_Dmote_proba']], how='inner', on='UniqueID')
                      

In [ ]:
test_unq_ids = df[df["isAttended"]=="???"]
test_unq_ids.shape

(128688, 66)

In [ ]:
ensemble_test.shape

(128688, 12)

In [ ]:
ensemble_test.rename(columns = {'CatBoost_model_preds':'Catboost',
                                      'model_preds':'NN',
                                      'XGB_model_preds':'XGB',
                                      'y_proba':'XGB_dummy',
                                      'Yamini_Dmote_proba':'Smote'}, inplace = True)

In [ ]:
ensemble_validation.head(1)

,UniqueID,isAttended_Yes,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LAR-417-6-20,1,OPEN,8.0,100.0,2.0,0.877009,0.86532,0.999988,0.873752,0.999957,0.865867


In [ ]:
ensemble_test.head(1)

,UniqueID,acct_id_aa,count,%Attended,event_name_count,class_name,NN,Catboost,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LV-101-10-1,AACLT21LV-101-10-1,54.0,94.444444,18.0,OPEN,0.913725,0.931003,0.975374,0.933363,0.97097,0.844401


In [ ]:
cols = ["UniqueID",	"class_name",	"count",	"%Attended",	"event_name_count",	"Catboost",	"NN",	"XGB",	"LGBM",	"LGBM_AccID",	"Smote"]
ensemble_test = ensemble_test[cols]

In [ ]:
ensemble_test.head(1)

,UniqueID,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LV-101-10-1,OPEN,54.0,94.444444,18.0,0.931003,0.913725,0.975374,0.933363,0.97097,0.844401


In [ ]:
ensemble_validation.head(2)

,UniqueID,isAttended_Yes,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LAR-417-6-20,1,OPEN,8.0,100.000000,2.0,0.877009,0.865320,0.999988,0.873752,0.999957,0.865867
1,CLT21PRE-644-19-16,0,OPEN,1216.0,87.171053,18.0,0.234931,0.242824,0.484125,0.277639,0.431342,0.208784


In [ ]:
labs = ensemble_validation["isAttended_Yes"].values
ensemble_validation = ensemble_validation.drop(["isAttended_Yes"], 1)
ensemble_validation["isAttended_Yes"] = labs

In [ ]:
ensemble_validation.head(2)

,UniqueID,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote,isAttended_Yes
0,CLT21LAR-417-6-20,OPEN,8.0,100.000000,2.0,0.877009,0.865320,0.999988,0.873752,0.999957,0.865867,1
1,CLT21PRE-644-19-16,OPEN,1216.0,87.171053,18.0,0.234931,0.242824,0.484125,0.277639,0.431342,0.208784,0


In [ ]:
X=ensemble_validation.iloc[:,:-1]
y = ensemble_validation.iloc[:, -1].values
X.head()

,UniqueID,class_name,count,%Attended,event_name_count,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,CLT21LAR-417-6-20,OPEN,8.0,100.000000,2.0,0.877009,0.865320,0.999988,0.873752,0.999957,0.865867
1,CLT21PRE-644-19-16,OPEN,1216.0,87.171053,18.0,0.234931,0.242824,0.484125,0.277639,0.431342,0.208784
2,CLT21LAR-411-9-1,OPEN,4.0,50.000000,2.0,0.862773,0.846407,0.504230,0.879845,0.498084,0.834916
3,CLT22JAX-417-1-9,OPEN,32.0,96.875000,16.0,0.868606,0.869788,0.972737,0.868594,0.972647,0.840681
4,CLT21SEA-423-7-6,OPEN,486.0,75.720165,9.0,0.740379,0.687670,0.787098,0.718494,0.770137,0.728897


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y,
                                                    stratify= ensemble_validation['isAttended_Yes'], 
                                                    test_size=0.1)

In [ ]:
train_ids = X_train["UniqueID"].values
train_classes = X_train["class_name"].values
train_counts = X_train["count"].values
train_att = X_train["%Attended"].values
train_event_cnt = X_train["event_name_count"].values

val_ids = X_val["UniqueID"].values
val_classes = X_val["class_name"].values
val_counts = X_val["count"].values
val_att = X_val["%Attended"].values
val_event_cnt = X_val["event_name_count"].values

test_ids = ensemble_test["UniqueID"].values
test_classes = ensemble_test["class_name"].values
test_counts = ensemble_test["count"].values
test_att = ensemble_test["%Attended"].values
test_event_cnt = ensemble_test["event_name_count"].values

X_train = X_train.drop(["UniqueID", "class_name", "count", "%Attended", "event_name_count"], 1)
X_val = X_val.drop(["UniqueID", "class_name", "count", "%Attended", "event_name_count"], 1)
X_test = ensemble_test.drop(["UniqueID", "class_name", "count", "%Attended", "event_name_count"], 1)

In [ ]:
X_train.head()

,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
149156,0.887697,0.894275,0.824568,0.886902,0.819239,0.878502
16294,0.896235,0.858507,0.733704,0.863960,0.763586,0.849502
203703,0.887989,0.885520,0.892226,0.894923,0.893729,0.879796
198869,0.924756,0.899301,0.981653,0.925008,0.978742,0.904242
229835,0.838051,0.853910,0.885082,0.850298,0.887258,0.834190


In [ ]:
X_val.head()

,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
116942,0.707954,0.757935,0.830096,0.728409,0.834541,0.707342
85306,0.859215,0.877295,0.910155,0.870380,0.918861,0.835305
169189,0.898337,0.883469,0.771149,0.903447,0.803385,0.897121
223456,0.010840,0.001104,0.002354,0.003995,0.002357,0.004236
202739,0.941635,0.914607,0.999980,0.941226,0.999958,0.926380


In [ ]:
X_test.head()

,Catboost,NN,XGB,LGBM,LGBM_AccID,Smote
0,0.931003,0.913725,0.975374,0.933363,0.970970,0.844401
1,0.934098,0.914444,0.876876,0.936630,0.873298,0.856253
2,0.934098,0.914444,0.876876,0.936630,0.873298,0.856253
3,0.935950,0.916711,0.933805,0.938848,0.917372,0.860317
4,0.923478,0.912708,0.887876,0.924761,0.876436,0.837479


In [ ]:
# scaler = StandardScaler()
# X_train_scaled_encoded = scaler.fit_transform(X_train)
# X_val_scaled_encoded = scaler.transform(X_val)

In [ ]:
def learning_rate_010_decay_power_099(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_010_decay_power_0995(current_iter):
    base_learning_rate = 0.1
    lr = base_learning_rate  * np.power(.995, current_iter)
    return lr if lr > 1e-3 else 1e-3

def learning_rate_005_decay_power_099(current_iter):
    base_learning_rate = 0.05
    lr = base_learning_rate  * np.power(.99, current_iter)
    return lr if lr > 1e-3 else 1e-3

In [ ]:
n_HP_points_to_test = 50

In [ ]:
fit_params={"early_stopping_rounds":30, 
            "eval_metric" : 'binary_logloss', 
            "eval_set" : [(X_val,[int(each) for each in y_val])],
            'eval_names': ['valid'],
            #'callbacks': [lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_099)],
            'verbose': 100,
            'categorical_feature': 'auto'}

In [ ]:
get_params_ = {'boosting_type': 'gbdt',
 'class_weight': None,
 'num_leaves': 8,
 'colsample_bytree': 0.6579,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 200,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 10000,
 'n_jobs': 4,
 'num_leaves': 31,
 'objective': None,
 'random_state': 314,
 'reg_alpha': 0.0,
 'reg_lambda': 10.0,
 'silent': True,
 'subsample': 0.9274,
 'subsample_for_bin': 200000,
 'subsample_freq': 0,
 'metric': 'None'}

In [ ]:
opt_parameters = {'colsample_bytree': 0.6579, 'min_child_samples': 322, 'min_child_weight': 0.001, 
                  'num_leaves': 8, 'reg_alpha': 0, 'reg_lambda': 50, 'subsample': 0.9274} 


In [ ]:
clf_final = lgb.LGBMClassifier(**get_params_)

#Train the final model with learning rate decay
clf_final.fit(X_train, [int(each) for each in y_train], **fit_params, callbacks=[lgb.reset_parameter(learning_rate=learning_rate_010_decay_power_0995)])


Training until validation scores don't improve for 30 rounds.
[100]	valid's binary_logloss: 0.292132
[200]	valid's binary_logloss: 0.291658
[300]	valid's binary_logloss: 0.291531
[400]	valid's binary_logloss: 0.291456
[500]	valid's binary_logloss: 0.291387
[600]	valid's binary_logloss: 0.291366
[700]	valid's binary_logloss: 0.291347
[800]	valid's binary_logloss: 0.291333
[900]	valid's binary_logloss: 0.291323
[1000]	valid's binary_logloss: 0.291318
[1100]	valid's binary_logloss: 0.291311
[1200]	valid's binary_logloss: 0.291308
Early stopping, best iteration is:
[1248]	valid's binary_logloss: 0.291306


LGBMClassifier(colsample_bytree=0.6579, metric='None', min_child_samples=200,
               n_estimators=10000, n_jobs=4, random_state=314, reg_lambda=10.0,
               subsample=0.9274)

In [ ]:
y_train = np.array([np.array(each) for each in y_train])
y_val = np.array([np.array(each) for each in y_val])

In [ ]:
threshold=0.5

In [ ]:
y_pred_val_logits = clf_final.predict_proba(X_val)
y_pred_val_logits = [each[1] for each in y_pred_val_logits]
y_pred_val = [1 if each>threshold else 0 for each in y_pred_val_logits]
print(classification_report([int(each) for each in y_val.tolist()], y_pred_val))


              precision    recall  f1-score   support

           0       0.85      0.57      0.68      5146
           1       0.89      0.97      0.93     18019

    accuracy                           0.88     23165
   macro avg       0.87      0.77      0.81     23165
weighted avg       0.88      0.88      0.87     23165



In [ ]:
y_pred_val_logits = clf_final.predict_proba(X_val)
y_pred_val_logits = [each[1] for each in y_pred_val_logits]
y_pred_val = [1 if each>threshold else 0 for each in y_pred_val_logits]
print(classification_report([int(each) for each in y_val.tolist()], y_pred_val))


              precision    recall  f1-score   support

           0       0.85      0.57      0.68      5146
           1       0.89      0.97      0.93     18019

    accuracy                           0.88     23165
   macro avg       0.87      0.77      0.80     23165
weighted avg       0.88      0.88      0.87     23165



In [ ]:
def get_heuristic_pred(row):
  if ((row['%Attended']<10) & (row['count']>=10) & (row["event_name_count"]>2)) or \
  (row["class_name"] in ['DIST-7HOLD','DIST-EXCP','DIST-WIP','KILL SEATS','OB2','OWNERSHIP','SINGLE GAME SRO','XTEST', 'UNTOUCHABLE','OBSTRUCTED']):
    return 0
  else:
    return row['model_pred']

In [ ]:
# val_pred_df = pd.DataFrame()
# val_pred_df["UniqueID"] = val_ids
# val_pred_df["count"] = val_counts
# val_pred_df["%Attended"] = val_att
# val_pred_df["event_name_count"] = val_event_cnt
# val_pred_df["class_name"] = val_classes
# val_pred_df["Actual"] = [int(each) for each in y_val.tolist()]
# val_pred_df["model_pred"] = y_pred_val
# val_pred_df["heuristic_preds"] = val_pred_df.apply(lambda x: get_heuristic_pred(x), 1)

In [ ]:
# print(classification_report([int(each) for each in y_val.tolist()], val_pred_df["heuristic_preds"].tolist()))


In [ ]:
y_pred_test = clf_final.predict(X_test)


In [ ]:
sub_df = pd.DataFrame()
sub_df["UniqueID"] = test_ids
sub_df["isAttended"] = y_pred_test

In [ ]:
sub_df.shape

(128688, 2)

In [ ]:
sub_df.head()

,UniqueID,isAttended
0,CLT21LV-101-10-1,1
1,CLT21LV-101-10-10,1
2,CLT21LV-101-10-11,1
3,CLT21LV-101-10-12,1
4,CLT21LV-101-10-13,1


In [ ]:
sub_df["isAttended"].value_counts()

1    113598
0     15090
Name: isAttended, dtype: int64

In [ ]:
sub_df.to_csv("/content/drive/MyDrive/Crossroads/predictions/Submission6_Ensemble.csv", index=False)